In [2]:
from nemreader import NEMFile
import pandas as pd
from zoneinfo import ZoneInfo

# Path to your NEM12 file
path = "./data/Sample2.csv"

# Parse file
nf = NEMFile(path)
df = nf.get_data_frame()  # → columns: nmi, suffix, serno, t_start, t_end, value, quality, evt_code, evt_desc

# --- Prepare Data ---
# Localise timestamps (AEST, no DST)
df["t_start"] = pd.to_datetime(df["t_start"]).dt.tz_localize(ZoneInfo("Australia/Brisbane"))

# Positive imports only (this script doesn’t flip export signs)
df["value"] = df["value"].astype(float).abs()

# Compute yearly usage (kWh)
df["year"] = df["t_start"].dt.year
yearly_usage = df.groupby("year")["value"].sum().reset_index(name="usage_kwh")

print(yearly_usage)


   year  usage_kwh
0  2015    997.776
1  2016    897.654


In [3]:
df["direction"] = df["suffix"].apply(lambda s: "export" if s == "B1" else "import")
summary = df.groupby(["year", "direction"])["value"].sum().reset_index()
print(summary)


   year direction    value
0  2015    import  997.776
1  2016    import  897.654
